# ECE9039 MACHINE LEARNING 

Temp:
- Features Extraction
- Autoencoders
- MLP
- CNN
- GANs
- Transfer Learning with Pre-trained Models
- Ensemble Learning

1. Accuracy
The most straightforward metric, it measures the proportion of correctly classified instances out of the total instances.

2. Precision
Precision is the ratio of correctly predicted positive observations to the total predicted positives. It is a measure of a classifier's exactness. High precision relates to a low false positive rate.

3. Recall (Sensitivity or True Positive Rate)
Recall is the ratio of correctly predicted positive observations to all observations in the actual class. It is a measure of a classifier's completeness.

4. F1 Score
The F1 Score is the harmonic mean of Precision and Recall. An F1 Score might be a better measure to use if you need to seek a balance between Precision and Recall and there is an uneven class distribution (large number of actual negatives).

5. Confusion Matrix
A confusion matrix is a table used to describe the performance of a classification model on a set of test data for which the true values are known. It allows you to visualize the performance of 

10. Top-k Accuracy
In cases where the model can predict multiple classes, the top-k accuracy might be used. It measures whether the correct label is within the top-k predicted labels

12. Cross-Entropy Loss
Also known as log loss, it measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label.

13. Balanced Accuracy
This metric is used for imbalanced datasets and is calculated as the average of recall obtained on each class. It compensates for the fact that traditional accuracy can be skewed when class distributions are imbalanced..an algorithm..teness.

## Data Preprocessing and Augmentation

In [25]:
import tensorflow as tf
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from collections import Counter
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageOps
import random

In [26]:
def load_images_and_labels(dataset_path, target_size=(224, 224)):
    images = []
    labels = []
    
    # Loop through each subfolder in the dataset directory
    for root, dirs, files in os.walk(dataset_path):
        if root == dataset_path:
            continue
        label = os.path.basename(root)
        count = 0
        for file in sorted(files):
            if file.lower().endswith(".jpg"):
                # Construct the full path to the image file
                file_path = os.path.join(root, file)
                image = Image.open(file_path).convert('RGB').resize(target_size)
                images.append(np.array(image))
                labels.append(label)
                count += 1
                
    return np.array(images), np.array(labels)

In [27]:
dataset_path = 'RealWaste/'
images, labels = load_images_and_labels(dataset_path)

print("Images shape:", images.shape)
print("Labels count:", len(np.unique(labels)))

Images shape: (4752, 224, 224, 3)
Labels count: 9


In [28]:
indices = np.arange(images.shape[0])
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

In [29]:
label_counts = Counter(labels)
for label, count in label_counts.items():
    print(f"{label}: {count}")

Metal: 790
Plastic: 921
Paper: 500
Cardboard: 461
Food Organics: 411
Glass: 420
Vegetation: 436
Miscellaneous Trash: 495
Textile Trash: 318


In [30]:
num_of_test_samples_per_class = 100
target_num_of_train_per_class = 800

In [31]:
classes = np.unique(labels)
test_indices = []
train_indices = []

for class_label in classes:
    # Find the indices of images belonging to the current class
    class_indices = np.where(labels == class_label)[0]
    
    # Shuffle the indices to ensure random selection
    np.random.shuffle(class_indices)
    test_indices.extend(class_indices[:num_of_test_samples_per_class])
    train_indices.extend(class_indices[num_of_test_samples_per_class:])

test_images = images[test_indices]
test_labels = labels[test_indices]
train_images = images[train_indices]
train_labels = labels[train_indices]

In [32]:
print(train_images.shape)
print(train_labels.shape)

(3852, 224, 224, 3)
(3852,)


In [33]:
images = None
labels = None

In [34]:
label_counts = Counter(train_labels)
for label, count in label_counts.items():
    print(f"{label}: {count}")

Cardboard: 361
Food Organics: 311
Glass: 320
Metal: 690
Miscellaneous Trash: 395
Paper: 400
Plastic: 821
Textile Trash: 218
Vegetation: 336


In [35]:
dict = {}
for i in label_counts:
    desired = target_num_of_train_per_class-label_counts[i]
    dict[i] = desired if desired > 0 else 0

In [36]:
print(dict)
print(sum(dict.values()))

{'Cardboard': 439, 'Food Organics': 489, 'Glass': 480, 'Metal': 110, 'Miscellaneous Trash': 405, 'Paper': 400, 'Plastic': 0, 'Textile Trash': 582, 'Vegetation': 464}
3369


In [37]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [38]:
augmented_images = []
augmented_labels = []

def generate_augmented_image(image, label): #,index
    for x_batch in datagen.flow(image.reshape((1,)+image.shape),batch_size=1): # ,save_to_dir='Preview', save_prefix=str(index), save_format='jpg'
        augmented_images.append(x_batch[0])
        augmented_labels.append(label)
        break


In [39]:
i = 0
while sum(dict.values()) > 0:
    j = i % len(train_images)
    if dict[train_labels[j]] > 0:
        dict[train_labels[j]]-=1 
        generate_augmented_image(train_images[j],train_labels[j])
    i+=1

In [40]:
print(np.array(augmented_images).shape)
print(np.array(augmented_labels).shape)

(3369, 224, 224, 3)
(3369,)


In [41]:
train_images = np.concatenate((train_images,augmented_images))
train_labels = np.concatenate((train_labels,augmented_labels))

In [42]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(7221, 224, 224, 3)
(7221,)
(900, 224, 224, 3)
(900,)


In [43]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [44]:
X_train = train_images
y_train = train_labels_encoded

X_val, X_test, y_val, y_test = train_test_split(test_images, test_labels_encoded, test_size=0.5, random_state=42, stratify=test_labels_encoded)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(7221, 224, 224, 3)
(7221,)
(450, 224, 224, 3)
(450,)
(450, 224, 224, 3)
(450,)


In [47]:
train_images = None
train_labels = None
train_labels_encoded = None
test_images = None
test_labels = None
test_labels_encoded = None

## Model

In [ ]:
model = Sequential([
    Conv2D(256, (3, 3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(9, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test), batch_size=32)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss function Plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()